# Chapter 5 exercises

In [206]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split

### Conceptual Problem 2h

In [207]:
rng = np.random.default_rng(10)
store = np.empty(10000)
for i in range(10000):
    store[i] = np.sum(rng.choice(100, replace=True) == 4) > 0
np.mean(store)

0.0089

In [208]:
1/100

0.01

## Applied Exercises


In [209]:
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm
np.random.seed(0)


### 5.

In Chapter 4, we used logistic regression to predict the probability of
default using income and balance on the Default data set. We will
now estimate the test error of this logistic regression model using the
validation set approach. Do not forget to set a random seed before
beginning your analysis.
#### (a)
Fit a logistic regression model that uses income and balance to predict default

In [210]:
Default = load_data('Default')
Default

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [211]:
design = MS(['balance', 'income']).fit(Default)
X = design.fit_transform(Default)
y = Default.default == 'Yes'
glm = sm.GLM(y,
             X,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-11.540500,0.435000,-26.544,0.0
balance,0.005600,0.000000,24.835,0.0
income,0.000021,0.000005,4.174,0.0


In [212]:
probs = results.predict(exog=X)
labels = np.array(['Yes']*10000)
labels[probs<=0.5]='No'
labels[:1000]

array(['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'No', 'No', 'No', 'No', 'No', 'No', 'No', 'N

In [213]:
confusion_table(labels, Default.default)

Truth,No,Yes
Predicted,,
No,9629,225
Yes,38,108


#### (b)

Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

Split the sample set into a training set and a validation set.

In [214]:
Default_train, Default_test = train_test_split(Default,
                                         test_size=5000,
                                         random_state=0)


II. Fit a multiple logistic regression model using only the train-
ing observations.

In [215]:
# hp_mm = MS(['horsepower'])
X_train = design.fit_transform(Default_train)
y_train = Default_train.default == 'Yes'
glm = sm.GLM(y_train,
             X_train,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-11.389600,0.635000,-17.935,0.000
balance,0.005600,0.000000,16.792,0.000
income,0.000016,0.000007,2.151,0.031


III. Obtain a prediction of default status for each individual in
the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

In [216]:
X_test = design.fit_transform(Default_test)
y_test = Default_test.default == 'Yes'
probs = results.predict(exog=X_test)

In [217]:
labels = np.array(['Yes']*5000)
labels[probs<=0.5] = 'No'
labels[:10]

array(['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No'],
      dtype='<U3')

In [218]:
from ISLP import confusion_table

In [219]:
confusion_table(labels, Default_test.default)

Truth,No,Yes
Predicted,,
No,4801,132
Yes,13,54


IV. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [220]:
(13+132)/5000

0.029

#### (c)

Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Com- ment on the results obtained.

In [221]:
Default_train, Default_test = train_test_split(Default,
                                         test_size=5000,
                                         random_state=1)
# hp_mm = MS(['horsepower'])
X_train = design.fit_transform(Default_train)
y_train = Default_train.default == 'Yes'
glm = sm.GLM(y_train,
             X_train,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)
X_test = design.fit_transform(Default_test)
y_test = Default_test.default == 'Yes'
probs = results.predict(exog=X_test)
labels = np.array(['Yes']*5000)
labels[probs<=0.5] = 'No'
labels[:10]
ct=confusion_table(labels, Default_test.default)
(ct.iloc[1,0]+ct.iloc[0,1])/5000


0.025

In [222]:
Default_train, Default_test = train_test_split(Default,
                                         test_size=5000,
                                         random_state=2)
# hp_mm = MS(['horsepower'])
X_train = design.fit_transform(Default_train)
y_train = Default_train.default == 'Yes'
glm = sm.GLM(y_train,
             X_train,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)
X_test = design.fit_transform(Default_test)
y_test = Default_test.default == 'Yes'
probs = results.predict(exog=X_test)
labels = np.array(['Yes']*5000)
labels[probs<=0.5] = 'No'
labels[:10]
ct=confusion_table(labels, Default_test.default)
(ct.iloc[1,0]+ct.iloc[0,1])/5000

0.0248

In [223]:
Default_train, Default_test = train_test_split(Default,
                                         test_size=200,
                                         random_state=2)
# hp_mm = MS(['horsepower'])
X_train = design.fit_transform(Default_train)
y_train = Default_train.default == 'Yes'
glm = sm.GLM(y_train,
             X_train,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)
X_test = design.fit_transform(Default_test)
y_test = Default_test.default == 'Yes'
probs = results.predict(exog=X_test)
labels = np.array(['Yes']*200)
labels[probs<=0.5] = 'No'
labels[:10]
ct=confusion_table(labels, Default_test.default)
(ct.iloc[1,0]+ct.iloc[0,1])/200

0.035

In [224]:
Default_train, Default_test = train_test_split(Default,
                                         test_size=200,
                                         random_state=1)
# hp_mm = MS(['horsepower'])
X_train = design.fit_transform(Default_train)
y_train = Default_train.default == 'Yes'
glm = sm.GLM(y_train,
             X_train,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)
X_test = design.fit_transform(Default_test)
y_test = Default_test.default == 'Yes'
probs = results.predict(exog=X_test)
labels = np.array(['Yes']*200)
labels[probs<=0.5] = 'No'
labels[:10]
ct=confusion_table(labels, Default_test.default)
(ct.iloc[1,0]+ct.iloc[0,1])/200

0.03

In [225]:
Default_train, Default_test = train_test_split(Default,
                                         test_size=200,
                                         random_state=0)
# hp_mm = MS(['horsepower'])
X_train = design.fit_transform(Default_train)
y_train = Default_train.default == 'Yes'
glm = sm.GLM(y_train,
             X_train,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)
X_test = design.fit_transform(Default_test)
y_test = Default_test.default == 'Yes'
probs = results.predict(exog=X_test)
labels = np.array(['Yes']*200)
labels[probs<=0.5] = 'No'
labels[:10]
ct=confusion_table(labels, Default_test.default)
(ct.iloc[1,0]+ct.iloc[0,1])/200

0.02

#### (d)

Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the val- idation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [226]:
Default.student = Default.student.map(dict(Yes=1, No=0))
Default

,default,student,balance,income
0,No,0,729.526495,44361.625074
1,No,1,817.180407,12106.134700
2,No,0,1073.549164,31767.138947
3,No,0,529.250605,35704.493935
4,No,0,785.655883,38463.495879
...,...,...,...,...
9995,No,0,711.555020,52992.378914
9996,No,0,757.962918,19660.721768
9997,No,0,845.411989,58636.156984
9998,No,0,1569.009053,36669.112365


In [227]:
design = MS(['balance', 'income', 'student']).fit(Default)
Default_train, Default_test = train_test_split(Default,
                                         test_size=5000,
                                         random_state=2)
X_train = design.fit_transform(Default_train)
y_train = Default_train.default == 'Yes'
glm = sm.GLM(y_train,
             X_train,
             family=sm.families.Binomial())
results = glm.fit()
X_test = design.fit_transform(Default_test)
probs = results.predict(exog=X_test)
labels = np.array(['Yes']*5000)
labels[probs<=0.5] = 'No'
ct=confusion_table(labels, Default_test.default)
(ct.iloc[1,0]+ct.iloc[0,1])/5000

0.0254

Adding the dummy variable for student seems to not affect the test error rate by much.

### 6.
We continue to consider the use of a logistic regression model to
predict the probability of default using income and balance on the
Default data set. In particular, we will now compute estimates for the
standard errors of the income and balance logistic regression coefcients in two diferent ways: (1) using the bootstrap, and (2) using the
standard formula for computing the standard errors in the sm.GLM()
function. Do not forget to set a random seed before beginning your
analysis

#### (a)
Using the summarize() and sm.GLM() functions, determine the
estimated standard errors for the coefcients associated with
income and balance in a multiple logistic regression model that
uses both predictors.

In [228]:
Default = load_data('Default')
Default

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [229]:
Default.default = Default.default.map(dict(Yes=1, No=0))
Default

,default,student,balance,income
0,0,No,729.526495,44361.625074
1,0,Yes,817.180407,12106.134700
2,0,No,1073.549164,31767.138947
3,0,No,529.250605,35704.493935
4,0,No,785.655883,38463.495879
...,...,...,...,...
9995,0,No,711.555020,52992.378914
9996,0,No,757.962918,19660.721768
9997,0,No,845.411989,58636.156984
9998,0,No,1569.009053,36669.112365


In [230]:
Default

,default,student,balance,income
0,0,No,729.526495,44361.625074
1,0,Yes,817.180407,12106.134700
2,0,No,1073.549164,31767.138947
3,0,No,529.250605,35704.493935
4,0,No,785.655883,38463.495879
...,...,...,...,...
9995,0,No,711.555020,52992.378914
9996,0,No,757.962918,19660.721768
9997,0,No,845.411989,58636.156984
9998,0,No,1569.009053,36669.112365


In [231]:
design = MS(['balance', 'income']).fit(Default)
X = design.fit_transform(Default)
y = Default.default == 1
glm = sm.GLM(y,
             X,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-11.540500,0.435000,-26.544,0.0
balance,0.005600,0.000000,24.835,0.0
income,0.000021,0.000005,4.174,0.0


The standard errors for the coefficients associated with income and balance are approximately 5*10^-6 and 0.

#### (b)

Write a function, boot_fn(), that takes as input the Default data
set as well as an index of the observations, and that outputs
the coefcient estimates for income and balance in the multiple
logistic regression model.

A bootstrap standard error function is below:

In [232]:
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

A bootstrap function for boothstrapping a regression model:

In [233]:
def boot_GLM(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return  sm.GLM(Y_,
             X_,
             family=sm.families.Binomial()).fit().params

The function that outputs the coefficient estimates for income and balance:

In [234]:
boot_fn=partial(boot_GLM, MS(['balance','income']), 'default')

In [235]:
rng = np.random.default_rng(0)
np.array([boot_fn(Default,
          rng.choice(1000,
                     1000,
                     replace=True)) for _ in range(10)])

array([[-1.04036736e+01,  4.98631061e-03,  2.03233678e-05],
       [-1.03639410e+01,  5.26184982e-03,  1.50556538e-05],
       [-1.19866812e+01,  6.12590454e-03,  3.34683163e-05],
       [-1.15513279e+01,  5.91434158e-03,  1.95389076e-05],
       [-1.55028507e+01,  7.00298761e-03,  7.88004295e-05],
       [-1.03744307e+01,  5.00180534e-03,  3.02375769e-05],
       [-1.22011845e+01,  5.76451958e-03,  3.39766209e-05],
       [-1.24689282e+01,  6.66585837e-03,  2.05016679e-05],
       [-1.04468359e+01,  4.73259586e-03,  3.16776354e-05],
       [-1.32518147e+01,  6.40224875e-03,  4.08903036e-05]])

In [236]:
np.asarray(Default)

array([[0, 'No', 729.5264952072861, 44361.62507426691],
       [0, 'Yes', 817.180406555498, 12106.1347003149],
       [0, 'No', 1073.54916401173, 31767.1389473999],
       ...,
       [0, 'No', 845.411989217448, 58636.1569838071],
       [0, 'No', 1569.0090533837197, 36669.1123645833],
       [0, 'Yes', 200.92218263479697, 16862.9523209407]], dtype=object)

#### Part (c)

In [237]:
boot_se=boot_SE(boot_fn,
                Default,
                B=1000,
                seed=10)

In [238]:
boot_se

intercept    0.425280
balance      0.000227
income       0.000005
dtype: float64

We now see a Standard Error for balance. This is probably more accurate.

Let's consider what happens when balance and income are scaled.

In [239]:
Default_copy=Default.copy(deep=True)

In [240]:
Default_copy.balance=Default_copy.balance*100
Default_copy

,default,student,balance,income
0,0,No,72952.649521,44361.625074
1,0,Yes,81718.040656,12106.134700
2,0,No,107354.916401,31767.138947
3,0,No,52925.060475,35704.493935
4,0,No,78565.588293,38463.495879
...,...,...,...,...
9995,0,No,71155.502049,52992.378914
9996,0,No,75796.291845,19660.721768
9997,0,No,84541.198922,58636.156984
9998,0,No,156900.905338,36669.112365


In [242]:
Default

,default,student,balance,income
0,0,No,729.526495,44361.625074
1,0,Yes,817.180407,12106.134700
2,0,No,1073.549164,31767.138947
3,0,No,529.250605,35704.493935
4,0,No,785.655883,38463.495879
...,...,...,...,...
9995,0,No,711.555020,52992.378914
9996,0,No,757.962918,19660.721768
9997,0,No,845.411989,58636.156984
9998,0,No,1569.009053,36669.112365


In [241]:
design_copy = MS(['balance', 'income']).fit(Default_copy)
X = design_copy.fit_transform(Default_copy)
y = Default_copy.default == 1
glm = sm.GLM(y,
             X,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-11.540500,0.435000,-26.544,0.0
balance,0.000056,0.000002,24.835,0.0
income,0.000021,0.000005,4.174,0.0


#### Part (d)

The assumption that they have the same variance and the fact that they are on different scales gives more accuracy to the bootstrap method's standard errors.

### 7.

### 8.

### 9.